In [3]:
from torch import Generator, use_deterministic_algorithms, tensor
from functools import partial
from matplotlib.pyplot import plot

from pathlib import Path
from karpathy_series.makemore.data import read_data
from karpathy_series.makemore.encoding.character import CharacterEncoder, CharacterSet, StringEncoder
from karpathy_series.makemore.training.data import TrainingSequencer, DataSplit
from karpathy_series.makemore.training.learning import Learner
from karpathy_series.makemore.models.generation import NGramNetGenerator
from karpathy_series.makemore.models.mlp import MPLNetComponents
from karpathy_series.makemore.bigrams import NGram

%matplotlib inline
use_deterministic_algorithms(True)
rand_generator = Generator()
seed = 2147483647

def create_generator():
    return rand_generator.manual_seed(seed)

In [2]:
data_path = Path("../../data/names.txt")
words = read_data(data_path)
char_set = CharacterSet.from_words(words)
char_encoder = CharacterEncoder.from_charset(char_set)
string_encoder = StringEncoder(char_encoder)

context_size = 4
embedding_dims = 10
hidden_dims = 200

n_gram_gen = partial(NGram.generate, context_size, char_set.pad) 
ts = TrainingSequencer(string_encoder, char_encoder, n_gram_gen)

In [12]:
import random
random.seed(42)
data_split = DataSplit.split(words, 80, 10, 10)
learning_sequence = ts.training_sequence(data_split.training, 32, True)
validation_set = ts.training_set(data_split.validation)

In [8]:
g = rand_generator.manual_seed(seed)
mpl_net = MPLNetComponents.init(4, char_encoder.size, embedding_dims, context_size, hidden_dims, g)
generator = NGramNetGenerator(char_set, string_encoder, char_encoder, mpl_net)
losses = []

In [11]:
print(sum(p.size() for p in mpl_net.layers), "parameters")
print("Layers:")
for k, p in enumerate(mpl_net.layers):
    print(k, p.describe())

136151 parameters
Layers:
0 Embedding [27, 10]
1 Flatten last 2 dims
2 Linear [40, 200] with bias
3 BatchNorm1d [200]
4 Functional
5 Linear [200, 200] with bias
6 BatchNorm1d [200]
7 Functional
8 Linear [200, 200] with bias
9 BatchNorm1d [200]
10 Functional
11 Linear [200, 200] with bias
12 BatchNorm1d [200]
13 Functional
14 Linear [200, 27] with bias
15 BatchNorm1d [27]


In [ ]:
learner = Learner(mpl_net, 0.1)
losses += learner(learning_sequence, epochs=50, report_epochs=10)

In [10]:
print(mpl_net.run(*validation_set))

tensor(3.6856, grad_fn=<NllLossBackward0>)


In [ ]:
for k in range(20):
    print(generator())

In [ ]:
plot(tensor(losses))